<h1>Fatal Shark Attack Predictor</h1>
In this notebook we will first load the cleaned data of the sharks attacks, we will generate an array of labeled points in order to predict if the attack of a shark can be fatal or not, according to the next variables:
<br>Activity performed
<br>Country
<br>Age
<br>Month
<br>Type activity
<br>Hour
<br>Sex

In [60]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pandas as pd

from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel
from pyspark.mllib.util import MLUtils

from ipywidgets import widgets
from IPython.display import display

from pyspark.mllib.regression import LabeledPoint, Vectors

conf = SparkConf().setAppName("appName")
sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=appName, master=local[*]) created by __init__ at <ipython-input-1-9f004c2d8128>:14 

In [2]:
df = pd.read_csv("output.csv")

data = sqlCtx.createDataFrame(df)

In [3]:
data.show()

+----------+-----------+-----------+----------+---------+-----+--------------+----+---------+
|Unnamed: 0|Clean Fatal|Activity Id|Country Id|Clean Age|Month|Id Type Attack|Hour|Sex Clean|
+----------+-----------+-----------+----------+---------+-----+--------------+----+---------+
|         0|          1|          0|         0|       16|    9|             1|  13|        0|
|         1|          1|          0|         0|       36|    9|             1|  11|        0|
|         2|          1|          0|         0|       43|    9|             1|  10|        0|
|         3|          1|          0|         1|       14|    9|             1|  13|        0|
|         4|          1|          0|         1|       14|    9|             1|  13|        0|
|         5|          1|          2|         1|       14|    9|             2|  13|        0|
|         6|          1|          5|         0|       60|    9|             1|  15|        0|
|         7|          1|          1|         0|       51|   

In [4]:
#Split data to two sets, one for testing and the other for training the Naive Bayes algorithm
training, test = data.randomSplit([0.6, 0.4])


def to_labeled_point(row):
    return LabeledPoint(row[1],[row[2], row[3], row[4], row[5], row[6], row[7],row[8]])

training_rdd = training.map(to_labeled_point)

print (training_rdd.collect())
test_rdd = test.map(to_labeled_point)


[LabeledPoint(1.0, [0.0,0.0,16.0,9.0,1.0,13.0,0.0]), LabeledPoint(1.0, [0.0,1.0,14.0,9.0,1.0,13.0,0.0]), LabeledPoint(1.0, [1.0,0.0,51.0,9.0,1.0,14.0,1.0]), LabeledPoint(2.0, [0.0,2.0,50.0,9.0,1.0,15.0,0.0]), LabeledPoint(1.0, [10.0,0.0,9.0,9.0,1.0,13.0,0.0]), LabeledPoint(1.0, [0.0,0.0,37.0,8.0,1.0,14.0,0.0]), LabeledPoint(1.0, [0.0,3.0,20.0,8.0,1.0,17.0,0.0]), LabeledPoint(1.0, [8.0,4.0,15.0,8.0,1.0,13.0,0.0]), LabeledPoint(1.0, [18.0,0.0,21.0,8.0,1.0,13.0,0.0]), LabeledPoint(1.0, [1.0,5.0,40.0,7.0,1.0,11.0,0.0]), LabeledPoint(1.0, [1.0,6.0,72.0,7.0,1.0,13.0,0.0]), LabeledPoint(1.0, [14.0,1.0,14.0,7.0,2.0,13.0,0.0]), LabeledPoint(1.0, [0.0,1.0,18.0,7.0,1.0,12.0,0.0]), LabeledPoint(1.0, [0.0,7.0,29.0,7.0,1.0,19.0,0.0]), LabeledPoint(1.0, [0.0,1.0,36.0,7.0,1.0,13.0,0.0]), LabeledPoint(1.0, [2.0,4.0,14.0,7.0,1.0,13.0,0.0]), LabeledPoint(1.0, [2.0,0.0,14.0,7.0,2.0,13.0,0.0]), LabeledPoint(1.0, [0.0,0.0,9.0,7.0,1.0,13.0,1.0]), LabeledPoint(1.0, [18.0,0.0,11.0,7.0,1.0,11.0,1.0]), LabeledPo

In [5]:
# Train a naive Bayes model.
model = NaiveBayes.train(training_rdd, 1.0)

In [6]:
def check_result(pair):
    return pair[0]==pair[1]

# Make prediction and test accuracy on the training set.
predictionAndLabel = test_rdd.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(check_result).count() / test_rdd.count()
print ("Accuracy of the model obtained on training data : ", accuracy, "\n")

Accuracy of the model obtained on training data :  0.7237894736842105 



In [10]:
# Let's query (predict) the most probable value for the first variable in a new instance:
tobj = Vectors.dense([1,1,21,4,1,13,0])
print ("Most Probable value for ", tobj, " : ", model.predict(tobj))

Most Probable value for  [1.0,1.0,21.0,4.0,1.0,13.0,0.0]  :  1.0


<H1>Predictor</H1>
Select the next variables for predict the fatality of a possible shark attack

<H1>Activity</H1>
<ul>

<li>0  SURFING</li>
<li>1  SWIMMING</li>
<li>2  FISHING</li>
<li>3  SPEARFISHIN</li>
<li>4  BATHING</li>
<li>5  WADING</li>
<li>6  DIVING</li>
<li>7  STANDING</li>
<li>8  SNORKELING</li>
<li>9  SCUBA DIVING</li>
<li>10 BODY BOARDING</li>
<li>11 TREADING WATER1</li>
<li>12 PEARL DIVING</li>
<li>13 BOOGIE BOARDING</li>
<li>14 KAYAKING</li>
<li>15 FREE DIVING</li>
<li>16 FELL OVERBOARD</li>
<li>17 WINFSURFING</li>
<li>18 OTHER</li>

</ul>

In [39]:
text = widgets.Text()
display(text)

In [41]:
activity = text.value
print (activity)

1


<H1>Country</H1>
<ul>
<li>1  USA             </li>            
<li>2  AUSTRALIA                 </li>  
<li>3  SOUTH AFRICA             </li>    
<li>4  PAPUA NEW GUINEA          </li>   
<li>5  NEW ZEALAND               </li>   
<li>6  BRAZIL                     </li>  
<li>7  BAHAMAS                   </li>    
<li>8  MEXICO                     </li>   
<li>9  ITALY                     </li>    
<li>10 FIJI                       </li>   
<li>11 PHILIPPINES                </li>   
<li>12 REUNION                    </li>   
<li>13 NEW CALEDONIA              </li>   
<li>14 MOZAMBIQUE                 </li>   
</ul>

In [42]:
text = widgets.Text()
display(text)

In [45]:
country = text.value
print (country)

2


<H1>Age</H1>

In [47]:
text = widgets.Text()
display(text)

In [48]:
age = text.value
print (age)

12


<H1>Month</H1>
Values 1<->12

In [50]:
text = widgets.Text()
display(text)

In [51]:
month = text.value
print (month)

4


<H1>Type Activity (Risk)</H1>
Default Unprovoked
<ul>
<li>1  Unprovoked</li>
<li>2  Boat</li>
<li>5  Provoked</li>
</ul>

In [52]:
text = widgets.Text()
display(text)

In [53]:
type_activity = text.value
print (type_activity)

4


<H1>Hour</H1>
Between 1h-24h

In [54]:
text = widgets.Text()
display(text)

In [55]:
hour = text.value
print (hour)

12


In [ ]:
<H1>Sex</H1>
Male 0
Female 1

In [56]:
text = widgets.Text()
display(text)

In [57]:
sex = text.value
print (sex)

1


In [59]:
pred = Vectors.dense([activity,country,age,month,type_activity,hour,sex])

if model.predict(tobj) == 1.0:
    print ("FATAL!")
else:
    print ("Don't worry, but be careful")

print ("Most Probable value for ", pred, " : ", model.predict(tobj))

FATAL!
Most Probable value for  [1.0,2.0,12.0,4.0,4.0,12.0,1.0]  :  1.0
